# Fusion Swin + ConvNeXt

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import accuracy_score, f1_score, classification_report

project_root = Path.cwd().parent.parent
MODELS_DIR = project_root / "models"

print("Environnement initialisé")

## 1. Chargement des prédictions

In [ ]:
from datetime import datetime
import glob

swin_files = sorted(glob.glob(str(MODELS_DIR / "swin" / "img_swin_probs_val_*.npy")))
convnext_files = sorted(glob.glob(str(MODELS_DIR / "convnext" / "convnext_probs_val_*.npy")))

if not swin_files:
    raise FileNotFoundError("Aucun fichier Swin trouve. Verifiez que le modele Swin a bien exporte les predictions.")
if not convnext_files:
    raise FileNotFoundError("Aucun fichier ConvNeXt trouve. Verifiez que le modele ConvNeXt a bien exporte les predictions.")

# Utiliser les fichiers les plus recents (dernier dans la liste triee)
swin_probs_file = swin_files[-1]
convnext_probs_file = convnext_files[-1]

# Extraire les timestamps des noms de fichiers
swin_timestamp = swin_probs_file.split("_val_")[-1].replace(".npy", "")
convnext_timestamp = convnext_probs_file.split("_val_")[-1].replace(".npy", "")

print(f"\nFichiers selectionnes:")
print(f"  Swin timestamp: {swin_timestamp}")
print(f"  ConvNeXt timestamp: {convnext_timestamp}")

# =========================================================================
# Chargement des predictions
# =========================================================================
print("\nChargement des predictions...")

# Swin
swin_probs = np.load(MODELS_DIR / "swin" / f"img_swin_probs_val_{swin_timestamp}.npy")
swin_labels = np.load(MODELS_DIR / "swin" / f"img_swin_labels_val_{swin_timestamp}.npy")
print(f"Swin charge: {swin_probs.shape}")

# ConvNeXt
convnext_probs = np.load(MODELS_DIR / "convnext" / f"convnext_probs_val_{convnext_timestamp}.npy")
convnext_labels = np.load(MODELS_DIR / "convnext" / f"convnext_labels_val_{convnext_timestamp}.npy")
print(f"ConvNeXt charge: {convnext_probs.shape}")

# Verifications
assert swin_probs.shape == convnext_probs.shape, f"Shapes incompatibles: {swin_probs.shape} vs {convnext_probs.shape}"
assert np.array_equal(swin_labels, convnext_labels), "Les labels de validation ne correspondent pas!"
labels = swin_labels

NUM_CLASSES = swin_probs.shape[1]
NUM_SAMPLES = swin_probs.shape[0]

print(f"\nEchantillons de validation: {NUM_SAMPLES:,}")
print(f"Classes: {NUM_CLASSES}")
print("Verification: Les deux modeles ont bien ete evalues sur les memes echantillons.")

## 2. Performance des modèles individuels

In [ ]:
# Swin v2
swin_preds = swin_probs.argmax(axis=1)
swin_acc = accuracy_score(labels, swin_preds)
swin_f1 = f1_score(labels, swin_preds, average='weighted')

print("Swin Transformer v2 (224×224):")
print(f"  Accuracy: {swin_acc*100:.2f}%")
print(f"  F1 Score: {swin_f1:.4f}")

# ConvNeXt
convnext_preds = convnext_probs.argmax(axis=1)
convnext_acc = accuracy_score(labels, convnext_preds)
convnext_f1 = f1_score(labels, convnext_preds, average='weighted')

print("\nConvNeXt (384×384):")
print(f"  Accuracy: {convnext_acc*100:.2f}%")
print(f"  F1 Score: {convnext_f1:.4f}")

best_single_f1 = max(swin_f1, convnext_f1)
best_single_name = "ConvNeXt" if convnext_f1 > swin_f1 else "Swin v2"
print(f"\nMeilleur modèle: {best_single_name} (F1 = {best_single_f1:.4f})")

## 3. Fusion: Moyenne Simple

In [ ]:
fusion_simple = (swin_probs + convnext_probs) / 2
fusion_simple_preds = fusion_simple.argmax(axis=1)

simple_acc = accuracy_score(labels, fusion_simple_preds)
simple_f1 = f1_score(labels, fusion_simple_preds, average='weighted')

print("Moyenne Simple:")
print(f"  Accuracy: {simple_acc*100:.2f}%")
print(f"  F1 Score: {simple_f1:.4f}")
print(f"  Amélioration: {(simple_f1 - best_single_f1)*100:+.2f}%")

## 4. Fusion: Moyenne Pondérée (recherche)

In [ ]:
print("Recherche du poids optimal (0 à 1, 101 étapes)...")
print("Formule: P_final = w * P_Swin + (1-w) * P_ConvNext\n")

weights = np.linspace(0, 1, 101)
best_weight = 0.0
best_f1 = 0.0
best_acc = 0.0

results = []
for w in weights:
    fusion_probs = w * swin_probs + (1 - w) * convnext_probs
    fusion_preds = fusion_probs.argmax(axis=1)
    
    acc = accuracy_score(labels, fusion_preds)
    f1 = f1_score(labels, fusion_preds, average='weighted')
    
    results.append({
        'weight': w,
        'swin_weight': w,
        'convnext_weight': 1 - w,
        'accuracy': acc,
        'f1_weighted': f1
    })
    
    if f1 > best_f1:
        best_f1 = f1
        best_acc = acc
        best_weight = w

print(f"Poids optimal trouvé: {best_weight:.2f}")
print(f"  - Contribution Swin: {best_weight:.1%}")
print(f"  - Contribution ConvNeXt: {(1-best_weight):.1%}")
print(f"\nPerformance:")
print(f"  Accuracy: {best_acc*100:.2f}%")
print(f"  F1 Score: {best_f1:.4f}")
print(f"  Amélioration: {(best_f1 - best_single_f1)*100:+.2f}%")

# Top 5
results_df = pd.DataFrame(results)
top5 = results_df.nlargest(5, 'f1_weighted')
print("\nTop 5 configurations:")
print(top5[['swin_weight', 'convnext_weight', 'accuracy', 'f1_weighted']].to_string(index=False))

## 5. Fusion: Moyenne Géométrique

In [ ]:
fusion_geometric = np.sqrt(swin_probs * convnext_probs)
fusion_geometric_preds = fusion_geometric.argmax(axis=1)

geometric_acc = accuracy_score(labels, fusion_geometric_preds)
geometric_f1 = f1_score(labels, fusion_geometric_preds, average='weighted')

print("Moyenne Géométrique:")
print(f"  Accuracy: {geometric_acc*100:.2f}%")
print(f"  F1 Score: {geometric_f1:.4f}")
print(f"  Amélioration: {(geometric_f1 - best_single_f1)*100:+.2f}%")

## 6. Fusion: Moyenne Harmonique

In [ ]:
eps = 1e-10
fusion_harmonic = 2 / ((1 / (swin_probs + eps)) + (1 / (convnext_probs + eps)))
fusion_harmonic_preds = fusion_harmonic.argmax(axis=1)

harmonic_acc = accuracy_score(labels, fusion_harmonic_preds)
harmonic_f1 = f1_score(labels, fusion_harmonic_preds, average='weighted')

print("Moyenne Harmonique:")
print(f"  Accuracy: {harmonic_acc*100:.2f}%")
print(f"  F1 Score: {harmonic_f1:.4f}")
print(f"  Amélioration: {(harmonic_f1 - best_single_f1)*100:+.2f}%")

## 7. Comparaison de toutes les méthodes

In [ ]:
print("="*70)
print("COMPARAISON DES MÉTHODES DE FUSION")
print("="*70)

comparison_data = [
    {'Méthode': 'Swin v2 (baseline)', 'Accuracy': f"{swin_acc*100:.2f}%", 
     'F1 Score': f"{swin_f1:.4f}", 'Amélioration': '-'},
    {'Méthode': 'ConvNeXt (baseline)', 'Accuracy': f"{convnext_acc*100:.2f}%", 
     'F1 Score': f"{convnext_f1:.4f}", 'Amélioration': '-'},
    {'Méthode': 'Moyenne Simple', 'Accuracy': f"{simple_acc*100:.2f}%", 
     'F1 Score': f"{simple_f1:.4f}", 'Amélioration': f"{(simple_f1 - best_single_f1)*100:+.2f}%"},
    {'Méthode': f'Moyenne Pondérée (w={best_weight:.2f})', 'Accuracy': f"{best_acc*100:.2f}%", 
     'F1 Score': f"{best_f1:.4f}", 'Amélioration': f"{(best_f1 - best_single_f1)*100:+.2f}%"},
    {'Méthode': 'Moyenne Géométrique', 'Accuracy': f"{geometric_acc*100:.2f}%", 
     'F1 Score': f"{geometric_f1:.4f}", 'Amélioration': f"{(geometric_f1 - best_single_f1)*100:+.2f}%"},
    {'Méthode': 'Moyenne Harmonique', 'Accuracy': f"{harmonic_acc*100:.2f}%", 
     'F1 Score': f"{harmonic_f1:.4f}", 'Amélioration': f"{(harmonic_f1 - best_single_f1)*100:+.2f}%"}
]

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

# Meilleure méthode
all_fusion_f1s = [simple_f1, best_f1, geometric_f1, harmonic_f1]
best_fusion_f1 = max(all_fusion_f1s)
best_method_idx = all_fusion_f1s.index(best_fusion_f1)
best_methods = ['Moyenne Simple', f'Moyenne Pondérée (w={best_weight:.2f})', 
                'Moyenne Géométrique', 'Moyenne Harmonique']

print(f"\n{'='*70}")
print(f"Meilleure méthode: {best_methods[best_method_idx]}")
print(f"  F1 Score: {best_fusion_f1:.4f}")
print(f"  Accuracy: {comparison_data[best_method_idx + 2]['Accuracy']}")
print(f"  Amélioration vs meilleur modèle: {(best_fusion_f1 - best_single_f1)*100:+.2f}%")
print("="*70)

## 8. Rapport de classification détaillé

In [ ]:
# Utiliser la meilleure méthode
if best_method_idx == 0:
    final_probs = fusion_simple
elif best_method_idx == 1:
    final_probs = best_weight * swin_probs + (1 - best_weight) * convnext_probs
elif best_method_idx == 2:
    final_probs = fusion_geometric
else:
    final_probs = fusion_harmonic

final_preds = final_probs.argmax(axis=1)

print(f"Rapport de classification - {best_methods[best_method_idx]}")
print("="*70)
print(classification_report(labels, final_preds, digits=4))

## 9. Sauvegarde des résultats

In [ ]:
import json
from datetime import datetime

# Timestamp pour cette execution de fusion
fusion_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

FUSION_DIR = MODELS_DIR / "Swin_ConvNext_Fusion"
FUSION_DIR.mkdir(parents=True, exist_ok=True)

# Sauvegarder les predictions avec timestamp
np.save(FUSION_DIR / f"img_fusion_probs_val_{fusion_timestamp}.npy", final_probs)
np.save(FUSION_DIR / f"img_fusion_labels_val_{fusion_timestamp}.npy", labels)
np.save(FUSION_DIR / f"img_fusion_preds_val_{fusion_timestamp}.npy", final_preds)

# Metadonnees avec timestamps des modeles sources
metadata = {
    "fusion_timestamp": fusion_timestamp,
    "swin_timestamp": swin_timestamp,
    "convnext_timestamp": convnext_timestamp,
    "model_1": "Swin Transformer v2 (224x224)",
    "model_2": "ConvNeXt (384x384)",
    "best_fusion_method": best_methods[best_method_idx],
    "best_weight": float(best_weight) if best_method_idx == 1 else None,
    "swin_f1": float(swin_f1),
    "convnext_f1": float(convnext_f1),
    "fusion_f1": float(best_fusion_f1),
    "improvement_absolute": float(best_fusion_f1 - best_single_f1),
    "improvement_relative_pct": float((best_fusion_f1 / best_single_f1 - 1) * 100),
    "validation_samples": int(NUM_SAMPLES),
    "num_classes": int(NUM_CLASSES)
}

metadata_filename = f"fusion_metadata_{fusion_timestamp}.json"
with open(FUSION_DIR / metadata_filename, 'w') as f:
    json.dump(metadata, f, indent=2)

comparison_filename = f"fusion_comparison_{fusion_timestamp}.csv"
comparison_df.to_csv(FUSION_DIR / comparison_filename, index=False)

print("="*70)
print("FICHIERS SAUVEGARDES")
print("="*70)
print(f"Repertoire: {FUSION_DIR}")
print(f"\nPredictions:")
print(f"  - img_fusion_probs_val_{fusion_timestamp}.npy")
print(f"  - img_fusion_labels_val_{fusion_timestamp}.npy")
print(f"  - img_fusion_preds_val_{fusion_timestamp}.npy")
print(f"\nMetadonnees:")
print(f"  - {metadata_filename}")
print(f"  - {comparison_filename}")
print("="*70)

## 10. Résumé final

In [ ]:
print("="*70)
print("RÉSUMÉ FINAL - FUSION SWIN + CONVNEXT")
print("="*70)

print(f"\nModèles fusionnés:")
print(f"  1. Swin Transformer v2 (224×224) - F1 = {swin_f1:.4f}")
print(f"  2. ConvNeXt (384×384)             - F1 = {convnext_f1:.4f}")

print(f"\nMéthodes de fusion testées:")
print(f"  1. Moyenne Simple:              F1 = {simple_f1:.4f}")
print(f"  2. Moyenne Pondérée (w={best_weight:.2f}):  F1 = {best_f1:.4f}")
print(f"  3. Moyenne Géométrique:         F1 = {geometric_f1:.4f}")
print(f"  4. Moyenne Harmonique:          F1 = {harmonic_f1:.4f}")

print(f"\nMeilleure méthode: {best_methods[best_method_idx]}")
print(f"  F1 Score:  {best_fusion_f1:.4f}")
print(f"  Accuracy:  {comparison_data[best_method_idx + 2]['Accuracy']}")
print(f"  Amélioration:")
print(f"    - Absolue: {(best_fusion_f1 - best_single_f1)*100:+.2f}%")
print(f"    - Relative: {(best_fusion_f1 / best_single_f1 - 1)*100:+.2f}%")

print(f"\nDonnées:")
print(f"  - Échantillons de validation: {NUM_SAMPLES:,}")
print(f"  - Classes: {NUM_CLASSES}")
print(f"  - Division: Splits unifiés du projet (data/splits/)")

print("="*70)